In [24]:
import pandas as pd
import numpy as np

# Import data
df = pd.read_csv("dummy_players.csv")

# Remove suffixes
suffixes = ['Sr.', 'Jr.', 'Mr.', 'Ms.', 'Miss']
df['Name'] = df['Name'].apply(lambda x: ' '.join(w for w in x.split() if w not in suffixes))

# Pre-Assigning players locked into teams by request
set_team_map = {
    'Alice Smith': 'Team 1'
}
df['Set Team'] = df['Name'].map(set_team_map)

# Team Framework
DivTeamCount = {
    'Division A': 2,  # 'Divison': Teams
}

DivTeamDetails = {
    'Division A': {'Team 1': 'Monday', 'Team 2': 'Friday'}
}

# Team Balancing
FinalTeams = {}
UnassignedPlayers = []
SummaryRecords = []
valid_days = {'monday', 'tuesday', 'wednesday', 'thursday', 'friday'}

for div in df['Division'].unique():

    DF_by_div = df[df['Division'] == div].copy()
    div_team_count = DivTeamCount[div]
    div_teams_days = DivTeamDetails[div]
    div_teams = list(div_teams_days.keys())

    Teams = {name: [] for name in div_teams}
    TeamSkill = {name: 0 for name in div_teams}
    TeamCount = {name: 0 for name in div_teams}

    # Handle fixed players first
    FixedFilteredDF = DF_by_div[DF_by_div['Set Team'].notna()].copy()
    for _, player in FixedFilteredDF.iterrows():
        if player['Set Team'] in Teams:
            Teams[player['Set Team']].append(player.to_dict())
            TeamSkill[player['Set Team']] += player['Skill']
            TeamCount[player['Set Team']] += 1
            DF_by_div = DF_by_div[DF_by_div['PlayerID'] != player['PlayerID']]

    # Determine fair team sizes
    TotalPlayerCount = len(FixedFilteredDF) + len(DF_by_div)
    MinTeamSize = TotalPlayerCount // div_team_count
    MaxTeamSize = MinTeamSize + 1

    # Sort remaining players by skill (strongest assigned first)
    DF_by_div = DF_by_div.sort_values(by='Skill', ascending=False)

    for _, player in DF_by_div.iterrows():
        # Normalize player availability
        player_days = set(d.strip().lower() for d in player['Days'].split(','))

        # Find eligible teams by matching availability
        EligibleTeams = [
            team for team in Teams
            if div_teams_days[team].strip().lower() in player_days
        ]

        if not EligibleTeams:
            UnassignedPlayers.append(player.to_dict())
            continue

        # Prefer the least-filled team, then lowest average skill
        EligibleTeams = sorted(EligibleTeams, key=lambda t: TeamCount[t])
        UncappedTeams = [t for t in EligibleTeams if TeamCount[t] < MaxTeamSize]

        if not UncappedTeams:
            UnassignedPlayers.append(player.to_dict())
            continue

        AssignedTeam = min(UncappedTeams, key=lambda t: TeamSkill[t])
        Teams[AssignedTeam].append(player.to_dict())
        TeamSkill[AssignedTeam] += player['Skill']
        TeamCount[AssignedTeam] += 1

    FinalTeams[div] = Teams

    # Build Summary
    for team_name, players in Teams.items():
        if players:
            avg_skill = round(sum(p['Skill'] for p in players) / len(players), 2)
            SummaryRecords.append({
                'Division': div,
                'Team': team_name,
                'Team Size': len(players),
                'Average Skill': avg_skill
            })

# Divisional Team Balancing Overview
summary_df = pd.DataFrame(SummaryRecords)
print("Summary:\n")
print(summary_df)

# Roster Output
print("\nTeam Rosters:\n")
for div, teams in FinalTeams.items():
    print(f"{div}")
    for team_name, players in teams.items():
        print(f"\n{team_name} (Players: {len(players)})")
        for p in players:
            print(f"  - {p['Name']} (Skill: {p['Skill']}, Availability: {p['Days']})")

Summary:

     Division    Team  Team Size  Average Skill
0  Division A  Team 1          2           3.00
1  Division A  Team 2          3           2.33

Team Rosters:

Division A

Team 1 (Players: 2)
  - Alice Smith (Skill: 3, Availability: Monday, Wednesday)
  - Evan Green (Skill: 3, Availability: Monday)

Team 2 (Players: 3)
  - Charlie Kim (Skill: 4, Availability: Monday, Friday)
  - Bob Jones (Skill: 2, Availability: Wednesday, Friday)
  - Diana Lopez (Skill: 1, Availability: Friday)
